# Project 5: Data Science & Machine Learning on Votings of the Swiss National Council

In project 5, we will analyze the voting behavior of the "Nationalrat" of the Swiss parliament in a number of ways. The project consists of 3 different files:

* Data Preparation (this notebook): Prepare the data for the other two notebooks.
* Predictions: Predict the voting behavior of individual members or the entire council.
* Unsupervised: Find lower-dimensional representations of the voting behavior and groups of members of parliament.

# Data Preparation

In this notebook, we clean and reformat the raw data on the votings of the national council analysis such that it will be ready for the other two notebooks. 

In most of the project (i.e., all except one part of the unsupervised learning notebook), we will consider the voting proposals as observational units, and the votes by the members of parliament as variables. With this understanding, additional information about the proposals are also variables added as a column. However, in a strict interpretation, this might not be fully compliant with the ideas of a tidy dataset, but it serves purpose as a joint basis for the supervised and unsupervised learning notebook. In the supervised learning notebook, we will in fact further transform the notebook into a fully tidy representation.

**You have to run this notebook before you can work on the other two.**

**To avoid potential issues with memory limitations (which might result in the kernel dying), we recomment that you click "Close and Shut Down Notebooks" (in the "File" tab) before you start another notebook.**

## Getting the Data
The voting behavior for every member of parliament as well as some information about the subjects of the vote are publicly available from https://www.parlament.ch/de/ratsbetrieb/abstimmungen/abstimmung-nr-xls (though only in German, French and Italian). We have downloaded the data for the summer sessions of the last four years. We will run the notebook on the latest data (i.e., from the year 2024) - but you are of course free to change to a different year, and you can also download further data files, namely from earlier years or from the council of states ("Ständerat").

In [ ]:
import pandas as pd
import numpy as np

The data is given as Excel sheets; we load it using `read_excel` from the `pandas` package. It might be worth to open the file in Excel to see how it's structured.

We see that the first two rows have a different format than the rest and do not contain substantial information. We therefore skip these two rows to avoid problems with the parsing of the file content:

In [ ]:
file_path = 'Abstimmungen_NR_2024SS_DE.xlsx'
df_votings_raw = pd.read_excel(file_path, header = None, skiprows=range(2))

## Format Headers and Columns
Let's have a look at the data we have just loaded:

In [ ]:
df_votings_raw.head(10)

In [ ]:
df_votings_raw.tail()

### Format Headers 
The data is formatted such that the actual voting data only starts at the line 8, with line 7 containing the column titles about the proposals. On the right of the information about the voting proposals, there is one column per member of parliament, containing some general information about that member (name, parliamentary group, canton, date of birth, date of swearing in).

In a transaction system (OLTP), one would have (at least) two different tables, one for the votes and one for the members of parliament. However, for now, we just combine the pieces of information on the members of parliament into one string that we will use as header. This is done in the next cell (you don't have to understand the details of this):

In [ ]:
header_rows = df_votings_raw.iloc[:8]
combined_headers = header_rows.apply(lambda x: x.ffill()).apply(lambda x: ' | '.join(x.dropna().astype(str)), axis=0)
print(combined_headers[:15])

We now build a data frame `df_votings` from the actual data items (i.e., the rows 8 and onwards) and use these combined strings as the new headers:

In [ ]:
# Combine headers into one (Ratsmitgliedes info)
df_votings = pd.DataFrame(df_votings_raw.values[8:], columns=combined_headers)
df_votings.head()

In [ ]:
df_votings.iloc[:5, :15]

### Drop Redundant Columns

 We drop the redundant columns `Ratsmitglied (Nr) | Name des Ratsmitgliedes | Rat | Fraktion | Kanton | Geburtsdatum | Vereidigungsdatum | Vereidigungsdatum` and `Rat` (which is `NR` for all items, as we only look at the national council). 

In [ ]:
# Remove unused columns
df_votings.drop(columns=['Ratsmitglied (Nr) | Name des Ratsmitgliedes | Rat | Fraktion | Kanton | Geburtsdatum | Vereidigungsdatum | Vereidigungsdatum',
                        'Rat', 'Teilnahme Präsident/in an der Abstimmung'], inplace = True)

### Translate Column Headers
Next, we translate the column headers to English

In [ ]:
df_votings.rename(columns={
    'Abstimmungsdatum': 'Voting Date', 
    'Zuständige Kommission': 'Responsible Commission',
    'Zuständige Behörde': 'Responsible Authority',
    'Geschäftsnummer': 'Topic Number',
    'Geschäftstitel': 'Topic Title',
    'Referenznummer': 'Reference ID',
    'Bedeutung Ja': 'Meaning of Yes',
    'Bedeutung Nein': 'Meaning of No',
    'Abstimmungsgegenstand': 'Voting Subject',
    'Vorlagetitel': 'Proposal Title'
}, inplace=True)

Furthermore, we set the "Referenznummer" of the proposal as index, as it is unique for each topic.

In [ ]:
df_votings.set_index('Reference ID', inplace=True)

**Exercise**: Adapt the above command to also translate the columns "Entscheid des Rates", "Anzahl 'Ja'", "Anzahl 'Nein'", "Anzahl Enthaltungen", "Anzahl 'entschuldigt'", "Anzahl 'nicht teilgenommen'" to "Council Decision", "Number of Yes", "Number of No", "Number of Abstentions", "Number of excused", "Number of non-participation".

## Overview of the Data
To verify the data is in good shape, we look at the different colums:

first 9 columns contain the proposal information:
- Voting Date
- Responsible Commission
- Responsible Authority
- Topic Number
- Topic Title
- Meaning of Yes
- Meaning of No
- Voting Proposal
- Voting Title

last 6 columns are summary of voting:
- Council Decision
- Number of Yes
- Number of No
- Number of Abstentions
- Number of excused
- Number of non-participation

The columns in between the votes by the members of parliament:
- 4049 | Aebischer, Matthias | NR | S | BE | 18.10.1967 | 04.12.2023 | 04.12.2023
- 10803 | Aellen, Cyril | NR | RL | GE | 29.02.1972 | 04.12.2023 | 04.12.2023
- \...
- 4179 | Zuberbühler, David | NR | V | AR | 20.02.1979 | 04.12.2023 | 04.12.2023
- 10822 | Zybach, Ursula | NR | S | BE | 29.08.1967 | 04.12.2023 | 04.12.2023

To verify, we briefly look at each of these groups of columns and print the column names:

In [ ]:
case_info_col_count = 9
case_info_cols = list(df_votings.columns[:case_info_col_count])
print(case_info_cols)

In [ ]:
summary_col_count = 6
summary_cols = list(df_votings.columns[-summary_col_count:])
print(summary_cols)

As there are 200 members of parliament, one of them is the president. We only print the first and last few to check:

In [ ]:
senators_cols = list(df_votings.columns[case_info_col_count:-summary_col_count])

In [ ]:
len(senators_cols)

In [ ]:
for mem in senators_cols[:3]:
    print(mem)

In [ ]:
for mem in senators_cols[-3:]:
    print(mem)

To check the data content, we print the unique text values in some important columns and will replace text values with numbers later.

In [ ]:
cols_to_transform_ja_nein = senators_cols + ['Council Decision']
np.unique(df_votings.loc[:,cols_to_transform_ja_nein].values)

In [ ]:
df_votings['Responsible Commission'].unique()

In [ ]:
df_votings['Responsible Authority'].unique()

Note that there are missing values in `Responsible Commission` and `Responsible Authority`. We fill them with `Unknown`, as we already have `Unknown` as another value. We first count how many records will be affected:

In [ ]:
df_votings[['Responsible Commission', 'Responsible Authority']].isna().sum()

In [ ]:
df_votings['Responsible Authority'] = df_votings['Responsible Authority'].fillna('Unknown')
df_votings['Responsible Commission'] = df_votings['Responsible Commission'].fillna('Unknown')

In [ ]:
df_votings

## Data Transformation
In order to process the data using different machine learning techniques, we need to process the actual values on our dataframe.

### Deriving new Attributes
One common step is to compute new attributes (or features). For example, we might want to know the percentage of members of parliament that voted 'Yes' to a given proposal:

**Exercise:** Write code to compute the percent of `yes` votes, and store this information as a new column called `Percent_Yes` in the dataframe `df_votings`.

In [ ]:
# df_votings['Percent_Yes'] = 

### Conversion to Numeric Values
In the following cell we convert text to numbers.

The possible values for votes are
- Ja
- Nein
- Enthaltung
- Hat nicht teilgenommen
- Entschuldigt gem. Art. 57 Abs. 4
- Die Präsidentin/der Präsident stimmt nicht

We convert "Ja" (yes) and "Nein" (no) to 1 and -1, respectively. We map all different reasons for non-participantion to 0.

In [ ]:
df_nr_votings = df_votings.copy()
mapping_ja_nein = {'Ja': 1, 'Nein': -1, 'Enthaltung': 0, 'Hat nicht teilgenommen': 0, 
                   'Entschuldigt gem. Art. 57 Abs. 4': 0,
                   'Die Präsidentin/der Präsident stimmt nicht':0 }
df_nr_votings.loc[:, cols_to_transform_ja_nein] = \
            df_nr_votings.loc[:, cols_to_transform_ja_nein].map(mapping_ja_nein.get)

Next we convert `Responsible Commission` and `Responsible Authority` to one-hot encodings. We can use the function `get_dummies` from the `pandas` library to do so. We start with the `Responsible Commission`:

In [ ]:
df_RC_OH_encoded = pd.get_dummies(df_nr_votings['Responsible Commission'],  dtype=int)

Let us look at how the first row was transformed:

In [ ]:
df_nr_votings['Responsible Commission'][0]

In [ ]:
df_RC_OH_encoded.iloc[0]

We see that we got many new columns, one for each unique value in the original column `df_nr_votings['Responsible Commission']`. In the first row, the resonsible commission was `' FK-NR | FK-SR | N/A-D-V | WBK-NR | WBK-SR'`. In the transformed dataframe, there is a 1 in the column corresponding to this commission, and all other columns are set to 0. This is why this type of representing categorical attributes is called *one-hot encoding*.

We do the same for the `'Responsible Authority'`.

**Exercise:** Convert `'Responsible Authority'` to one-hot encoding.

In [ ]:
# df_RA_OH_encoded = ...

Next, we combine the original dataframe with the two newly created dataframes containing the one-hot encodings.

**Exercise:** Write code to combine the three dataframes into a single dataframe called `df_nr_votings`. Since we have replaced the two columns `'Responsible Commission'` and `'Responsible Authority'`, by one-hot encodings, we also want to drop the two original columns

In [ ]:
# df_nr_votings = ...

In [ ]:
df_nr_votings.head(5)

### Handling Missing Values
Finally, we have to check whether there are any missing values. 

Missing values are typically indicated as `NA`. However, some missing values might also be more difficult to find, as they might be encoded in a different way. In our dataframe, there are some records where the `Topic Number` is set to `'Unknown'`.

**Exercise:** Implement the following steps to get rid of missing values:
* Count the total number of `na` values. Remember that you can use the functions `isna()` in any dataframe to find out, for every cell of the dataframe, whether the corresponding value is `NA` or not. Furthermore, you can use the function `df.sum()` to sum over the first dimension of a dataframe `df`.
* Delete all records (rows) which contain a `NA` value.
* Identify (e.g., print out) the records that have an `Unknown` `Topic Number`. Decide on what to do with these.

### Vectorizing Text Data
Some of the columns contain text data. We will vectorize these texts using count vectorizers to get a representation that we will use afterwards.

The following block of code runs a joint `CountVectorizer` for the `text_columns`. Using a `for` loop, it then iterates over the columns and does a vectorization. 
* In the first line of the `for` loop, the count vectorizer is used to transform the data. This yields a matrix (`transformed_data`) where each row corresponds to a voting proposal, and each column corresponds to the number of times the respective word occurrs in that text column.
* In the second column, a `DataFrame` is generated from the matrix `transformed_data`, with the column name representing first the name of the column of the original data frame, and then, separated by a `_`, the word that is counted in the respective column of the output dataframe.
* Finally, in the third line of the `for` loop, the new data frame is combined with the previously generated text representations in `df_nr_vectorized_text_info`.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Identify columns that contain text
text_columns = ['Topic Title', 'Meaning of Yes', 'Meaning of No', 'Voting Subject', 'Proposal Title']

df_nr_vectorized_text_info = pd.DataFrame({})

# Transform each text column using CountVectorizer
for col in text_columns:
    # Initialize CountVectorizer
    vectorizer = CountVectorizer(max_features=50)
    # adapt the vectorizer to the dataset (i.e., the column we are currently considering
    transformed_data = vectorizer.fit_transform(df_nr_votings[col])
    # convert the data to an understandable dataframe
    transformed_df = pd.DataFrame(transformed_data.toarray(), columns=[f"{col}_{word}" for word in vectorizer.get_feature_names_out()], 
                                 index=df_nr_votings.index)
    df_nr_vectorized_text_info = pd.concat([df_nr_vectorized_text_info, transformed_df], axis = 1)

The `df_nr_vectorized_text_info` now contains all the proposals as rows, and all the information of the text columns as columns.

**Exercise:** While the technical aspect of the above code cell is nontrivial (but also not needed for the rest of the project), it's important you understand the result of this. Take a moment to understand the representation we have generated. For example, look at `df_nr_vectorized_text_info` or the intermediate results in `transformed_data` and `transformed_df`. 

**Optional Exercise**: In the above cell, we have not used any stop-words. When looking at `df_nr_vectorized_text_info`, you will find both very common words and numbers being counted (and the counts being represented in the columns of `df_nr_vectorized_text_info`). 

Processing text data is often tricky and can be tedious, but also can have a significant impact on the performance. Play around with the preprocessing, e.g. with the following modifications (and combinations thereof):
* `CountVectorizer()` takes an optional argument `stop_words` and a list of words to be ignored (typically, because they are considered very common and thus uninformative). For example, you can write `vectorizer = CountVectorizer(stop_words = ['der', 'die', 'das'])`. Add your own stopwords, or find a list of common words in German (unfortunately, `scikit-learn` has a predefined list of stopwords only for English).
* You can replace any number by `NUM` by replacing `df_nr_votings[col]` (in the first line of the `for` loop) by `df_nr_votings[col].replace('\d+', 'NUM', regex=True)`; the vectorizer will then only see `NUM` instead of any number. Alternatively, you can use `df_nr_votings[col].replace('\d+', '', regex=True)` to remove any number in the texts.

Finally, we make make a copy of the original `df_nr_votings` dataframe where we remove the text columns. We save this as `df_nr_votings_numeric`:

In [ ]:
df_nr_votings_numeric = df_nr_votings.copy().drop(columns=text_columns)

Let's look at the data frame `df_nr_votings_numeric`:

In [ ]:
df_nr_votings_numeric

It contains two columns with general information, then the cast votes of every member of parliament (one column for each of the 200 members), and then some more information about the proposal. To avoid confusion, we separate the cast votes from the information about the subject:

In [ ]:
df_nr_cast_votes = df_nr_votings_numeric.copy().iloc[:, 2:202]
df_nr_numeric_info = df_nr_votings_numeric.drop(columns=df_nr_cast_votes.columns)

`df_nr_numeric_info` still contains a column `Voting Date`, which is not numeric, and which we will not use in our analysis. We therefore drop it as well:

In [ ]:
df_nr_numeric_info.drop(columns='Voting Date', inplace=True)

## Data Overview
To summarize, we now have the following three dataframes:
* `df_nr_numeric_info`: Numeric information about the voting proposals, e.g., the responsible administration.
* `df_nr_vectorized_text_info`: vectorized text information about the voting proposals.
* `df_nr_cast_votes`: the votes by each member of parliament (each column corresponds to a member of parliament).
In all 3 dataframes, each line is one voting proposal.

In [ ]:
df_nr_numeric_info

In [ ]:
df_nr_vectorized_text_info

In [ ]:
df_nr_cast_votes

Finally, as we will train some models using all available data (i.e., the numerical and the text data), we combine `df_nr_numeric_info` and `df_nr_vectorized_text_info` to `df_nr_all_info`:

In [ ]:
df_nr_all_info = pd.concat([df_nr_numeric_info, df_nr_vectorized_text_info], axis=1)
df_nr_all_info.head()

In [ ]:
df_nr_all_info.describe()

## Save the datasets

Finally we are ready to save the dataframes into files that we will load when running  other analysis steps.

In [ ]:
df_nr_numeric_info.to_csv(file_path.replace('.xlsx', '_numeric_info.csv'))
df_nr_vectorized_text_info.to_csv(file_path.replace('.xlsx', '_vectorized_text_info.csv'))
df_nr_cast_votes.to_csv(file_path.replace('.xlsx', '_cast_votes.csv'))
df_nr_all_info.to_csv(file_path.replace('.xlsx', '_all_info.csv'))

Furthermore, for later reference, we save a brief summary for every voting proposal:

In [ ]:
info_cols = [ x for x in (case_info_cols + summary_cols + ['Percent_Yes']) if not x in ['Responsible Commission', 'Responsible Authority'] ]

In [ ]:
df_nr_votings[ info_cols ].to_csv(file_path.replace('.xlsx', '_summary.csv'))

**Exercise:**
* Given the dataset that we have now cleaned, what applications could you imagine? How could they be useful?
* What other data might be useful? What additional analysis could you do with additional data?

Discuss your ideas with another participant or somebody from the teaching team.